In [1]:
import pandas as pd

In [8]:
df_dpm = pd.read_csv('dpm-experiments.csv')
df_dfm = pd.read_csv('dfm-opt-experiments.csv')

In [9]:
def get_groups(x: str):
    y = x.split('-')[1]
    return ((int(y) - 1) // 10) + 1


df_dpm.insert(0, 'group', df_dpm['name'].apply(get_groups))
df_dfm.insert(0, 'group', df_dfm['name'].apply(get_groups))

In [10]:
def process_metrics(df_out: pd.DataFrame) -> pd.DataFrame:
    df_out["lb1_ratio"] = df_out["lb1_start"] / df_out["lower_bound"]
    df_out["lb5_ratio"] = df_out["lb5_start"] / df_out["lower_bound"]
    df_out["solved"] = (df_out["incumbent"] - df_out["lower_bound"]) ** 2 <= 1e-3


process_metrics(df_dpm)
process_metrics(df_dfm)

In [11]:
df_dpm.head()

,group,name,warmstart,lb1_start,lb5_start,incumbent,lower_bound,execution_time,gap,nodes,sequence,lb1_ratio,lb5_ratio,solved
0,1,dpm-001-20-5x6,1465,1326,1326,1381,1381,0.288417,0.0,12255,"[17, 10, 15, 2, 16, 19, 6, 18, 14, 12, 9, 5, 1...",0.960174,0.960174,True
1,1,dpm-002-20-5x6,1455,1328,1328,1385,1385,0.160825,0.0,9840,"[6, 3, 14, 8, 19, 0, 16, 1, 7, 11, 9, 5, 12, 4...",0.958845,0.958845,True
2,1,dpm-003-20-5x6,1559,1470,1470,1485,1485,0.011168,0.0,77,"[1, 16, 6, 7, 11, 17, 12, 3, 2, 14, 8, 4, 0, 1...",0.989899,0.989899,True
3,1,dpm-004-20-5x6,1519,1413,1413,1458,1458,0.121580,0.0,7669,"[9, 19, 12, 10, 2, 0, 15, 18, 4, 5, 7, 8, 3, 6...",0.969136,0.969136,True
4,1,dpm-005-20-5x6,1511,1371,1382,1465,1465,402.908985,0.0,21733552,"[0, 1, 2, 8, 12, 4, 16, 9, 10, 14, 15, 19, 13,...",0.935836,0.943345,True


In [12]:
def agg_dataframe(df_out: pd.DataFrame) -> pd.DataFrame:
    result = df_out.groupby('group', as_index=False).agg({
        'lb1_ratio': 'mean',
        'lb5_ratio': 'mean',
        'gap': 'mean',
        'execution_time': 'median',
        'nodes': 'median',
        'solved': 'sum',
    }).rename(columns={'Instance': 'Instances'})

    # Round lb ratios to 3 decimals
    result['lb1_ratio'] = result['lb1_ratio'].round(3)
    result['lb5_ratio'] = result['lb5_ratio'].round(3)

    # Convert gap to percentage
    result['gap'] = (result['gap'] * 100).round(2)

    # Round execution time to 4 significant digits
    result['execution_time'] = result['execution_time'].apply(
        lambda x: float(f"{x:.2g}")
    )

    # Format nodes in scientific notation with 2 significant digits
    result['nodes'] = result['nodes'].apply(
        lambda x: f"{x:.2g}"
    )

    # Insert group info
    result.insert(1, "n", [20, 20, 20, 50, 50, 50])
    result.insert(2, "m", [7, 10, 15, 7, 10, 15])

    return result

In [13]:
agg_dataframe(df_dfm)

,group,n,m,lb1_ratio,lb5_ratio,gap,execution_time,nodes,solved
0,1,20,7,0.964,0.970,0.00,0.055,2.9e+03,10
1,2,20,10,0.915,0.923,0.00,1.800,6e+04,10
2,3,20,15,0.882,0.908,0.73,260.000,5e+06,9
3,4,50,7,0.986,0.987,0.06,0.340,7.2e+03,9
4,5,50,10,0.980,0.983,0.59,790.000,1.9e+07,8
5,6,50,15,0.982,0.984,5.72,3600.000,3.7e+07,0


In [39]:
agg_dataframe(df_dpm)

,group,n,m,lb1_ratio,lb5_ratio,gap,execution_time,nodes,solved
0,1,20,7,0.966,0.971,0.00,0.22,1.1e+04,10
1,2,20,10,0.926,0.942,0.00,5.90,2.2e+05,10
2,3,20,15,0.890,0.920,0.85,350.00,6.2e+06,9
3,4,50,7,0.988,0.991,0.00,0.98,3.4e+04,10
4,5,50,10,0.983,0.989,0.32,560.00,1.5e+07,7
5,6,50,15,0.986,0.996,5.28,3600.00,4.4e+07,0


In [40]:
agg_dataframe(df_dpm).to_csv("dpm-opt-summary.csv", index=False)

In [41]:
agg_dataframe(df_dfm).to_csv("dfm-opt-summary.csv", index=False)

In [12]:
df_dpm.head()

,group,name,warmstart,lb1_start,lb5_start,incumbent,lower_bound,execution_time,gap,nodes,sequence,Group
0,1,dpm-001-20-5x6,1465,1326,1326,1381,1381,0.288417,0.0,12255,"[17, 10, 15, 2, 16, 19, 6, 18, 14, 12, 9, 5, 1...",1
1,1,dpm-002-20-5x6,1455,1328,1328,1385,1385,0.160825,0.0,9840,"[6, 3, 14, 8, 19, 0, 16, 1, 7, 11, 9, 5, 12, 4...",1
2,1,dpm-003-20-5x6,1559,1470,1470,1485,1485,0.011168,0.0,77,"[1, 16, 6, 7, 11, 17, 12, 3, 2, 14, 8, 4, 0, 1...",1
3,1,dpm-004-20-5x6,1519,1413,1413,1458,1458,0.121580,0.0,7669,"[9, 19, 12, 10, 2, 0, 15, 18, 4, 5, 7, 8, 3, 6...",1
4,1,dpm-005-20-5x6,1511,1371,1382,1465,1465,402.908985,0.0,21733552,"[0, 1, 2, 8, 12, 4, 16, 9, 10, 14, 15, 19, 13,...",1
